## This is the feedforward neuronal model

In [1]:
from foundations.helpers import scale_input_theory
from brian2 import *
from foundations.helpers import scale_to_freq
from foundations.make_dynamic_experiments import make_dynamic_experiments
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from models.currentmodel import Barrel_PC, Barrel_IN
from foundations.MI_calculations import analyze_exp
from foundations.helpers import make_spiketrain
%matplotlib inline

INFO       Cache size for target "cython": 1155 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\laure\.cython\brian_extensions" directory. [brian2]


ModuleNotFoundError: No module named 'foundations.MI_calculations'

In [8]:
#parameters setting 
#TODO: set these parameters in a file
baseline = 0
theta = 0
factor_ron_roff = 2
tau_PC = 250
ron_PC = 1./(tau_PC*(1+factor_ron_roff))
roff_PC = factor_ron_roff*ron_PC
mean_firing_rate_PC = (0.1)/1000
duration = 100000
tau_IN = 50
ron_IN = 1./(tau_IN*(1+factor_ron_roff))
roff_IN = factor_ron_roff*ron_IN
mean_firing_rate_IN = (0.5)/1000
sampling_rate = 5
dt = 1/sampling_rate
qon_qoff_type = 'balanced'
Er_exc, Er_inh = (0, -75)
target_PC = 1.4233
target_IN = 6.6397
on_off_ratio = 1.5
scale_list = np.append([1], np.arange(2.5, 302.5, 2.5))
scales = {'CC_PC':19, 'DC_PC':30, 'CC_IN':17, 'DC_IN':6}
N_runs = 1
defaultclock.dt = dt*ms
PC_i = 35
IN_i = 11

In [9]:
#Getting the input
[input_theory, dynamic_theory, hidden_state] = make_dynamic_experiments(qon_qoff_type, baseline, tau_PC, factor_ron_roff, mean_firing_rate_PC, sampling_rate, duration)
inp = [scale_input_theory(input_theory, 'current', 0, scales['CC_PC'], dt),scale_input_theory(input_theory, 'current', 0, scales['CC_PC'], dt)]

In [10]:
#This one is for the difference in view
start_scope()

inj_input = inp[0]
PC = Barrel_PC(dt=dt, amount = 2)
inj_input = inp[1]
IN = Barrel_IN(dt=dt)

PC_neuron = PC.getNeurongroup()
IN_neuron = IN.getNeurongroup()

syn = Synapses(IN_neuron, PC_neuron, on_pre='''v += 10*mV''', namespace = param)
syn.connect(i=0, j=0)

PC_M = StateMonitor(PC_neuron, 'v', record=True)
IN_M = StateMonitor(IN_neuron, 'v', record=True)
run(duration*ms)

NameError: name 'param' is not defined

In [ ]:
figure(figsize=(4, 8))
subplot(211)
plot(PC_M.t/ms, PC_M.v[1], label='pre_PC')
plot(IN_M.t/ms, IN_M.v[0], label="pre_IN")
legend()

subplot(212)
plot(PC_M.t/ms, PC_M.v[1], label='pre_PC')
plot(PC_M.t/ms, PC_M.v[0], label="post_PC")
legend(loc='best')
xlabel('Time (ms)');

In [ ]:
#This one is for the result and the SpikeMonitor
start_scope()

inj_input = inp[0]
PC = Barrel_PC(dt=dt)
inj_input = inp[1]
IN = Barrel_IN(dt=dt)

PC_neuron = PC.getNeurongroup()
IN_neuron = IN.getNeurongroup()

syn = Synapses(IN_neuron, PC_neuron, on_pre='''v += 10*mV''', namespace = param)
syn.connect(i=0, j=0)

PC_M = StateMonitor(PC_neuron, 'v', record=syn)
IN_M = StateMonitor(IN_neuron, 'v', record=True)

PC_S = SpikeMonitor(PC_neuron)
IN_S = SpikeMonitor(IN_neuron)

run(duration*ms)

In [ ]:
IN_spike = make_spiketrain(IN_S,duration,dt)
PC_spike = make_spiketrain(PC_S,duration,dt)

In [ ]:
result_PC = analyze_exp(ron_PC, roff_PC, hidden_state, input_theory, dt, 0, PC_spike)

In [ ]:
result_PC

In [34]:
result_PC = analyze_exp(ron_PC, roff_PC, hidden_state, input_theory, dt, 0, PC_spike)

In [5]:
result_PC.MI

NameError: name 'result_PC' is not defined

In [ ]:
R

In [ ]:
figure(figsize=(4, 8))
subplot(211)
plot(PC_M.t/ms, PC_M.v[1], label='pre_PC')
plot(IN_M.t/ms, IN_M.v[0], label="pre_IN")
legend()

subplot(212)
plot(PC_M.t/ms, PC_M.v[1], label='pre_PC')
plot(PC_M.t/ms, PC_M.v[0], label="post_PC")
legend(loc='best')
xlabel('Time (ms)');

In [5]:
start_scope()
net = Network()

#Initialising the neuron classes
PC = Barrel_PC(dt=dt)
IN = Barrel_IN(dt=dt)
#Getting the neurongroups
PC_neuron = PC.getNeurongroup()
PC_neuron_ref = PC.getNeurongroup() #making a reference neuron 
IN_neuron = IN.getNeurongroup()
#adding the neurons to the network
net.add(PC_neuron, IN_neuron, PC_neuron_ref)

#getting the statemonitors from befor the connection 
PC_M_ref = StateMonitor(PC_neuron_ref, 'v', record=True)
IN_M = StateMonitor(IN_neuron, 'v', record=True)
net.add(PC_M_ref, IN_M)
#making the synapse

param = PC.getNamespace().update(IN.getNamespace()) #getting all the parameters for in the synapse
syn = Synapses(IN_neuron, PC_neuron, on_pre='''v += 10*mV''', namespace = param)

syn.connect()

PC_M = StateMonitor(PC_neuron, 'v', record=syn)
net.add(syn, PC_M)

run(1*second)


In [ ]:
figure(figsize=(4, 8))
subplot(211)
plot(PC_M_ref.t/ms, PC_M_ref.v[0], label='pre_PC')
plot(IN_M.t/ms, IN_M.v[0], label="pre_IN")
legend()

subplot(212)
plot(PC_M_ref.t/ms, PC_M_ref.v[0], label='pre_PC')
plot(PC_M.t/ms, PC_M.v[0], label="post_PC")
legend(loc='best')
xlabel('Time (ms)');

In [ ]:
figure(figsize=(4, 8))
subplot(211)
plot(PC_M_ref.t/ms, PC_M_ref.v[0], label='pre_PC')
plot(IN_M.t/ms, IN_M.v[0], label="pre_IN")
legend()

subplot(212)
plot(PC_M_ref.t/ms, PC_M_ref.v[0], label='pre_PC')
plot(PC_M.t/ms, PC_M.v[0], label="post_PC")
legend(loc='best')
xlabel('Time (ms)');